In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from utils.orventro import deviation_metric_optimized, dev_metric
from datetime import datetime
from utils.transform_data import transform

xgb.set_config(verbosity=0)

In [2]:
train_df = pd.read_csv('./data/train.csv', index_col='id')
target = 'per_square_meter_price'

/home/orventro/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def prep(df):
    df = df.drop(['city', 'osm_city_nearest_name', 'date'], axis=1)
    df['floor'] = np.where(pd.to_numeric(df['floor'], errors='coerce').isnull().to_numpy(), 
                           -1.0, 
                           df['floor'].to_numpy())
    regions = df['region'].unique()
    reg_dict = dict(zip(regions, range(len(regions))))
    print(reg_dict)
    df['region'] = [reg_dict[r] for r in df['region']]
    df['street'] = [s if type(s) == float else float(s[1:]) for s in df['street']]
    return df.astype(float)

In [4]:
# dfm, dfp, dfo = transform(train_df)
train_df = prep(train_df)
# valid = train_df.iloc[100000:]
train = train_df

In [5]:
dfm.head()

,city,floor,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
id,,,,,,,,,,,,,,,,,,,,,
COL_4,Москва,-1,55.802590,37.487110,0,0,0,0,0,0,...,7.263889,5.684211,1963.229167,1960.500000,4,60.2,1338.0,2020-01-05,10,0
COL_22,Москва,-1,55.787457,37.609140,0,0,0,0,0,0,...,9.226950,9.857143,1963.276596,1961.142857,4,15997.0,18334.0,2020-01-05,100,0
COL_53,Москва,3.0,55.750366,37.585260,0,0,0,0,0,0,...,6.613027,6.547945,1932.019157,1925.712329,4,164.0,14144.0,2020-01-05,100,0
COL_54,Москва,4.0,55.749626,37.732574,0,0,0,0,0,0,...,4.000000,NaN,1929.000000,NaN,4,18.6,1101.0,2020-01-05,100,0
COL_55,Москва,-1.0,55.773026,37.589275,0,0,0,0,0,0,...,7.273973,7.500000,1942.392694,1946.055556,4,116.0,265.0,2020-01-05,10,0


In [5]:
def test_model1(model, train, valid):
    model.fit(train.drop([target], axis=1), train[target])
    pred = model.predict(valid.drop([target], axis=1))
    err = deviation_metric_optimized(pred, valid[target].to_numpy())
    print(model, err)

In [21]:
def train_stack(train):
    weights = train['price_type'].to_numpy() * 99 + 1
#     model_cat = CatBoostRegressor(iterations=100, verbose=0)
#                                  loss_function=deviation_metric_optimized,
#                                  eval_metric=deviation_metric_optimized)
    model_xgb = XGBRegressor(device='GPU')#, obj=dev_metric)
    model_lgb = LGBMRegressor(device='GPU')#, objective=dev_metric)
    level0 = list()
#     level0.append(('cat', model_cat))
    level0.append(('xgb', model_xgb))
    level0.append(('lgb', model_lgb))

    model_logreg = LinearRegression()

    model = StackingRegressor(estimators=level0, 
                              final_estimator=model_logreg, 
                              cv=5)
    model.fit(train.drop([target], axis=1), 
              train[target],
              sample_weight=weights)
    return model

def validate_stack(stack, valid):
    pred = stack.predict(valid.drop([target], axis=1))
    weights = valid['price_type'] * 99 + 1
    return deviation_metric_optimized(valid[target], pred, weights)

def predict(stack, save):
    test_df = pd.read_csv('./data/test.csv', index_col='id')
    test_df = prep(test_df)
    
    pred = pd.DataFrame(index=test_df.index)
    pred.insert(0, target, stack.predict(test_df)/8) #WILL REMOVE
    if save:
        pred.to_csv(f'./out_{datetime.now().time()}.csv')
    
    out_df = pd.read_csv('./out.csv', index_col='id')
    print(deviation_metric_optimized(pred[target], out_df[target]))

In [11]:
stack = train_stack(train)

In [12]:
# err = validate_stack(stack, valid)
# print(err)

In [22]:
predict(stack, False)

3.6816240454948934


In [ ]:
xgb.set_config(verbosity=1)
model_xgb = XGBRegressor(device='GPU')#, obj=dev_metric)
test_model1(model_xgb, train, valid)